In [1]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

In [2]:
# Librerias de uso común
import numpy as np
import torch
from scipy.spatial import distance
import os, json, cv2, random, shutil
import matplotlib.pyplot as plt
#from google.colab.patches import cv2_imshow

# Utilidades del detectron2

from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import ColorMode
from detectron2 import structures
from detectron2.structures.boxes import Boxes
from detectron2 import model_zoo
import detectron2.data.transforms as T
from detectron2.data import DatasetMapper, build_detection_train_loader   # the default mapper

In [3]:
register_coco_instances("dataset_train", {}, "Dataset/train.json", "Dataset/data")
register_coco_instances("dataset_test", {}, "Dataset/test.json", "Dataset/data")

In [4]:
MetadataCatalog.get("dataset_test")

namespace(name='dataset_test',
          json_file='Dataset/test.json',
          image_root='Dataset/data',
          evaluator_type='coco')

In [5]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("dataset_train",)
cfg.DATASETS.TEST = ("dataset_test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 250
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
cfg.TEST.DTECTIONS_PER_IMAGE = 20

In [6]:
dataloader = build_detection_train_loader(cfg,
   mapper=DatasetMapper(cfg, is_train=True, augmentations=[
      # T.Resize((391, 417))
   ]))

[01/11 18:00:23 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: []
WARNING [01/11 18:00:23 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[01/11 18:00:23 d2.data.datasets.coco]: Loaded 22 images in COCO format from Dataset/train.json
[01/11 18:00:23 d2.data.build]: Removed 0 images with no usable annotations. 22 images left.
[01/11 18:00:23 d2.data.build]: Distribution of instances among all 2 categories:
|  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|
|   diente   | 12           |    raiz    | 11           |
|            |              |            |              |
|   total    | 23           |            |              |
[01/11 18:00:23 d2.data.build]: Using training sampler TrainingSampler
[01/11 18:00:23 d2.data.common]: Serializing 22 elements to byte tensors and concatenating them all ...
[01/11 18:00:23 d2.data.common]: S

In [7]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)  # load last checkpoint or MODEL.WEIGHTS
trainer.train()

[01/11 18:00:26 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

2023-01-11 18:00:26.287726: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-11 18:00:26.430941: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/asl1014/miniconda3/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda-9.0/lib64/:/usr/local/cuda-9.0/extras/CUPTI/lib64/:
2023-01-11 18:00:26.430958: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-11 18:00:26.878294: W tensorflow/compiler/xla/stream_executor/platform/de

[01/11 18:00:27 d2.engine.train_loop]: Starting training from iteration 0


/home/asl1014/miniconda3/lib/python3.9/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/home/asl1014/miniconda3/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180549130/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[01/11 18:00:31 d2.utils.events]:  eta: 0:00:45  iter: 19  total_loss: 1.685  loss_cls: 0.8246  loss_box_reg: 0.1419  loss_mask: 0.6888  loss_rpn_cls: 0.006268  loss_rpn_loc: 0.0152  time: 0.1976  data_time: 0.0134  lr: 1.9231e-05  max_mem: 2335M
[01/11 18:00:35 d2.utils.events]:  eta: 0:00:41  iter: 39  total_loss: 1.293  loss_cls: 0.4545  loss_box_reg: 0.1297  loss_mask: 0.6685  loss_rpn_cls: 0.005339  loss_rpn_loc: 0.01548  time: 0.1974  data_time: 0.0030  lr: 3.9211e-05  max_mem: 2335M
[01/11 18:00:39 d2.utils.events]:  eta: 0:00:37  iter: 59  total_loss: 1.003  loss_cls: 0.1963  loss_box_reg: 0.1239  loss_mask: 0.6128  loss_rpn_cls: 0.005606  loss_rpn_loc: 0.01464  time: 0.1967  data_time: 0.0037  lr: 5.9191e-05  max_mem: 2335M
[01/11 18:00:43 d2.utils.events]:  eta: 0:00:33  iter: 79  total_loss: 0.9288  loss_cls: 0.1524  loss_box_reg: 0.1379  loss_mask: 0.5754  loss_rpn_cls: 0.009221  loss_rpn_loc: 0.0136  time: 0.1954  data_time: 0.0044  lr: 7.9171e-05  max_mem: 2335M
[01/11 18

In [11]:
# Look at training curves in tensorboard:
import tensorflow as tf
import datetime, os

%load_ext tensorboard
%tensorboard --logdir output --port 6007


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
